In [3]:
import random
import re
import requests
import time
import math
import numpy as np
import operator
from bs4 import BeautifulSoup as bs
from collections import Counter
import os
import json
def removeTag(soup,tag):
    [x.extract() for x in soup.select(tag)]

In [4]:
data = {}
for fname in os.listdir('./Workspace/data/openrice'):
    if 'openrice' in fname:
        with open('./Workspace/data/openrice'+'/'+fname, 'r') as f:
            jd = json.load(f)
            for ele in jd:
                if ele not in data:
                    data[ele] = jd[ele]
print len(data)

91822


In [5]:
urlList = []
for u in data.values():
    urlList.append(u['shortenUrl'])
print len(urlList)
print urlList[0]

91822
http://s.openrice.com/cHKSnW2OiO500


In [20]:
domain = 'http://tw.openrice.com'
rs = requests.session()
rs.headers.update({
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    })

def getRivewURL(rs, u):
    res = rs.get(u)
    soup = bs(res.text, 'html.parser')
    review = domain + soup.select('.main-menu.table-center li')[1].select('a')[0]['href']
    number = soup.select('.main-menu.table-center li')[1].select('a')[0].text
    count = int(re.search('(\d+)', number).group(1))
    page = int(math.ceil(count / 15.0))
    return (review, page, count)
#print getRivewURL(rs, 'http://tw.openrice.com/zh/taipei/r-叁和院-大安區-台灣菜-精緻料理-r249160')

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
def getContent(u, p):
    result = []
    driver = webdriver.Firefox()
    url = u
    driver.get(url)
    for i in xrange(1,p+1):
        element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".js-layout-option.full"))
        )
        res =  driver.page_source.encode('utf-8')
        soup = bs(res, 'html.parser')
        n = len(soup.select('.middle-header'))
        time.sleep(0.5)
        driver.find_element_by_css_selector(".js-layout-option.full").click()
        res = driver.page_source
        soup = bs(res, 'html.parser')
        for t in soup.select('.review-container'):
            temp = re.sub('<img.*>','',t.text)
            s = "".join(temp.split()).strip()
            result.append(s)
        if i < p:
            nextpage = driver.find_element_by_css_selector(".pagination-button.next.js-next")
            nextpage.click()
        else:
            driver.quit()
    return result
def getall(u):
    st = []
    review = getRivewURL(rs, u)
    st.extend(getContent(review[0],review[1]))
    return st

In [26]:
import time
start = time.time()
url = 'http://tw.openrice.com/zh/taipei/r-叁和院-大安區-台灣菜-精緻料理-r249160'
text = getall(url)
print time.time()-start

47.4827342033


In [57]:
text = []
for url in urlList:
    text.extend(getall(url))

http://s.openrice.com/cHKSnG2V3W600


KeyboardInterrupt: 